RNN

- 영화 리뷰 감정 분석과 기계번역시에 단어의 특징만 잡는 것이 아닌, 순서가 주는 정보까지 인지하는 형태의 신경망 재현

- 기존의 CNN 등의 모델은 사진 같은 정적 데이터를 인지한다. 현실에서는 순차적으로 경험하는 동적 데이터가 있다.
- 데이터가 순서대로 나열된 데이터( 순차적 데이터, 시계열 데이터)의 정보를 받아 전체 내용을 학습하는 RNN
- RNN은 정해지지 않은 길이의 배열을 읽고 설명하는 신경망.
- RNN은 시계열 데이터 정보를 하나씩 이력 받을 때마다 지금까지 입력된 벡터들을 종합해 은닉 벡터를 만들어 낸다. 

  ( 마지막으로 만들어진 은틱벡터K는 배열 속 모든 벡터들의 내용을 압축한 벡터라고 할 수 있다. )
 
- RNN계열의 신경망은 대표적으로 텍스트와 자연어를 처리하고 학습하는데 주로 이용. ( LSTM, GRU의 응용 RNN 개발 ) ->감정분석, 언어모델링, 기계번역에 이용
- 데이터 순서 정보 학습하는 점에서 RNN은 CIFAR-10같은 정적 데이터보다는 동영상, 자연어, 주가 등 동적 데이터 이용시 성능 극대화

In [6]:
#RNN을 이용해 텍스트 감정 분석 ( 영화 리뷰 감정 분석)
#IMDB영화리뷰 데이터 이용(5만건) 부정리뷰 1, 긍정리뷰 2로 레이블링
#리뷰의 긍정여부 판단하는 분류 모델 생성

#라이버르리 임포트
import os 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets

#하이퍼파라미터 정의
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 40
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

#데이터셋 로드, 텐서 변환

#텍스트->텐서변환 설정
TEXT = data.Field(sequential = True, batch_first = True, lower = True) #시퀀셜은 순차적인 데이터인지? 배치는 텐서 첫 입력차원 값이 배치사이즈인지?, lower로 모든 알파벳 소문자로 
LABEL = data.Field(sequential = False, batch_first = True)

trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

#임베딩용 단어사전 구축
TEXT.build_vocab(trainset, min_freq = 5) #최소 5번 이상 등장한 단어만 사전에 포함 // 5번 미만은 unk 토큰으로 대체 
LABEL.build_vocab(trainset)

#검증 데이터셋의 부족으로 학습셋을 스플릿 하여 이용 (+배치 단위로 쪼개서 학습 진행)
#데이터 스플릿
trainset, valset = trainset.split(split_ratio = 0.8)
#반복시 배치 생성 반복자 생성(iterator)
train_iter, val_iter, test_iter = data.BucketIterator.splits((trainset, valset, testset), batch_size = BATCH_SIZE, shuffle=True, repeat = False)

#사전속 단어, 레이블 수 정하는 변수 생성
vocab_size = len(TEXT.vocab)
n_classes = 2

#확인
print('학습셋 : {}  // 검증셋 : {}  //  테스트셋 : {} //  단어수 : {}  // 클래스 : {}'.format(len(trainset), len(valset), len(testset), vocab_size, n_classes))

학습셋 : 20000  // 검증셋 : 5000  //  테스트셋 : 25000 //  단어수 : 46159  // 클래스 : 2


In [7]:
#모델 생성 BasicGRU

class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
        super(BasicGRU, self).__init__()
        print('Building Basic GRU model')
        #은닉 벡터의 층은 복잡한 모델이 아닌 이상 2이하로 정의하는게 대부분
        self.n_layers = n_layers
        #임베딩은 모든 단어 수, 임베딩된 단어 텐서가 지니는 차원값을 입력받는다
        self.embed = nn.Embedding(n_vocab, embed_dim)
        #은닉 벡터 차원값, 드롭아웃 정의
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        #RNN모델 정의 RNN의 응용모델인 GRU를 사용하는 이유는 RNN은 입력이 길어지면 학습 도중 기울기가 너무 작거나 커져 앞부분의 정보를 정확히 담지 못할 수 있다.
        #이를 경사도 폭발, 소실이라 하며 이를 보완하기위한 대표적인 것이 GRU이다. 
        #GRU는 시계열 데이터 속 벡터 사이의 정보 전달량을 조절함으로써 기울기를 적정하게 유지하고 문장 앞부분의 정보가 끝까지 도달할 수 있게 도와준다. 
        #GRU는 업데이트 게이트(이전 은닉벡터가 지닌 정보를 새로운 은닉벡터가 얼마나 유지할지 결정), 리셋게이트(새로운 입력이 이전 은닉벡터와 어떻게 조합하는지 결정)가 있다. 
        self.gru = nn.GRU(embed_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True)
        #문장 전체 맥락을 담은 텐서를 신경망에 통과시켜 클래스에 대한 예측 출력 
        self.out= nn.Linear(self.hidden_dim, n_classes)
        
    #forward 함수 구현
    def forward(self, x):
        x = self.embed(x)
        #보통신경망과 달리 은닉벡터를 정의해 x와 함께 입력해줘야한다. 
        h_0 = self._init_state(batch_size = x.size(0))#은닉벡터
        x, _ = self.gru(x, h_0) #은닉벡터의 시계열 배열 형태 반환 -> 배치사이즈, 입력 x길이, hidden_dim의 모양을 지닌 3d텐서
        h_t = x[:,-1,:]#배치 내 모든 시계열 은닉 벡터들의 마지막 토큰들을 내포한 (batch_size, 1, hidden_dim)모양의 텐서 추출 // 영화 리뷰 배열 압축 은닉벡터
        self.dropout(h_t) # 드롭아웃 설정
        logit = self.out(h_t) #신경망에 입력하여 결과 출력
        return logit
    
    # 파라미터 함수가 신경망 모듈의 가중치 정모를 반복자 형태로 반환 ( 원소들을 각 실제 신경망의 가중치 텐서를 지닌 객체)
    def  _init_state(self, batch_size = 1):
        weight = next(self.parameters()).data # 첫 가중치 텐서 추출
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_() #모델 가중치와 같은 모양으로 텐서를 변환 후, 제로 호출해 텐서 내 모든값 0으로 초기화
    #첫 은닉 벡터(h_0)는 보통 모든 특성값이 0인 벡터로 설정

In [10]:
#학습, 평가 구현

def train(model, optimizer, train_iter):
    model.train()
    #반복바다 배치 데이터 반환 , 영화평 데이터와 그에 상응하는 레이블은 batch.text, batch.label을 통해 접근가능
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) #1과 2의 레이블값을 0과 1로 변환
        optimizer.zero_grad() #매번 기울기를 새로 계산하여 기울기를 0으로 초기화
        logit = model(x)
        
        #예측과 실제 레이블간의 오차를 구하고 기울기 계싼 후 최적화 과정의 반복
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()
        
        

In [11]:
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter :
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) #레이블 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = 'sum') # 각 배치의 평균이 아닌 데이터셋 전체 오차의 합
        total_loss += loss.item() #오차합 누적
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum() #모델이 맞힌 수를 담는다.
    size = len(val_iter.dataset)
    avg_loss = total_loss/size #오차합과 corrects변수를 데이터셋 크기로 나누어 오차 평군과, 정확도 평균 반환
    avg_accuracy = 100.0 *corrects / size
    return avg_loss, avg_accuracy

In [12]:
#모델 객체 정의 
#은닉벡터 차원값 256  // 임베딩 토큰의 차원값 128로 임의 설정
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
#최적화 알고리즘은 속도가 빠른 Adam이용
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

Building Basic GRU model


In [14]:
#학습실행 
best_val_loss = None

for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)
    
    print("EPOCSH : {}  // Val_loss : {}  //  val_accuracy : {}%".format(e, val_loss, val_accuracy))
    
    #검증오차가 최소인 최적모델 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir('snapshot'):
            os.makedirs('snapshot')
        torch.save(model.state_dict(), './snapshot/txtxclassification.pt')
        best_val_loss = val_loss
        
#본인은 GPU가 지원이 안되 CPU로 매우 오랜시간이 걸린다. 
#GPU(CUDA)를 이용가능한지 그래픽카드 제조사 홈페이지에서 확인 후 실행하는 것을 권장
#CPU 분석환경이 속도가 너무 느려 16epoch까지 진행후 학습 중단

EPOCSH : 1  // Val_loss : 0.6924577838897705  //  val_accuracy : 52.34000015258789%
EPOCSH : 2  // Val_loss : 0.6896089431762695  //  val_accuracy : 53.7400016784668%
EPOCSH : 3  // Val_loss : 0.6723749813079833  //  val_accuracy : 58.439998626708984%
EPOCSH : 4  // Val_loss : 0.3665471423625946  //  val_accuracy : 85.19999694824219%
EPOCSH : 5  // Val_loss : 0.36541864686012265  //  val_accuracy : 85.45999908447266%
EPOCSH : 6  // Val_loss : 0.3321242066502571  //  val_accuracy : 87.45999908447266%
EPOCSH : 7  // Val_loss : 0.35852700452804565  //  val_accuracy : 87.05999755859375%
EPOCSH : 8  // Val_loss : 0.38771652791500094  //  val_accuracy : 86.9800033569336%
EPOCSH : 9  // Val_loss : 0.3848258887052536  //  val_accuracy : 87.23999786376953%
EPOCSH : 10  // Val_loss : 0.4082783898353577  //  val_accuracy : 87.08000183105469%
EPOCSH : 11  // Val_loss : 0.44137678174972533  //  val_accuracy : 86.5%
EPOCSH : 12  // Val_loss : 0.46287915616035463  //  val_accuracy : 87.19999694824219

KeyboardInterrupt: 

In [15]:
#학습 마친 후, 테스트셋으로 모델 성능 시험(검증 오차가 최소화된 모델)
model.load_state_dict(torch.load('./snapshot/txtxclassification.pt'))
test_loss , test_acc = evaluate(model, test_iter)
print('테스트 오차 : {}  // 테스트 정확도 : {}'.format(test_loss, test_acc))

테스트 오차 : 0.3235560657119751  // 테스트 정확도 : 86.63200378417969
